In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# https://www.faa.gov/airports/airport_safety/airportdata_5010/menu/
# Converted from xls to csv
# Information Current as of 07/19/2018
airportData = pd.read_csv('NfdcFacilities.csv') 

/Users/Spencer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
airportData.head()

,SiteNumber,Type,LocationID,EffectiveDate,Region,DistrictOffice,State,StateName,County,CountyState,...,AirportPositionSourceDate,AirportElevationSource,AirportElevationSourceDate,ContractFuelAvailable,TransientStorage,OtherServices,WindIndicator,IcaoIdentifier,MinimumOperationalNetwork,Unnamed: 103
0,50009.*A,AIRPORT,'ADK,7/19/18,AAL,NONE,AK,ALASKA,ALEUTIANS WEST,AK,...,00:00.0,3RD PARTY SURVEY,00:00.0,NaN,HGR,CARGO,Y,PADK,NaN,N
1,50016.1*A,AIRPORT,'AKK,7/19/18,AAL,NONE,AK,ALASKA,KODIAK ISLAND,AK,...,00:00.0,NACO,00:00.0,NaN,NaN,NaN,Y,PAKH,NaN,N
2,50017.*A,AIRPORT,'Z13,7/19/18,AAL,NONE,AK,ALASKA,BETHEL,AK,...,00:00.0,STATE,00:00.0,NaN,NaN,CARGO,Y-L,NaN,NaN,N
3,50017.1*C,SEAPLANE BASE,'KKI,7/19/18,AAL,NONE,AK,ALASKA,BETHEL,AK,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N
4,50020.*A,AIRPORT,'AKI,7/19/18,AAL,NONE,AK,ALASKA,BETHEL,AK,...,00:00.0,STATE,00:00.0,NaN,NaN,CARGO,Y-L,PFAK,NaN,N


In [4]:
n = 0
for i in range(len(airportData)):
    if airportData['LocationID'][i][0]=="'":
        n+=1
if n==len(airportData):
    print("There're {} entries in aiportData. Each entry in LocationID column has a leading apostrophe.\nAll will be omitted when inserting values into our database table.".format(n))

There're 19615 entries in aiportData. Each entry in LocationID column has a leading apostrophe.
All will be omitted when inserting values into our database table.


In [5]:
south = 0
north = 0
west = 0
east = 0
for i in range(len(airportData)):
    if airportData['Type'][i]=='AIRPORT':
        if airportData['ARPLatitudeS'][i][-1]=='S':
            south+=1
        else:
            north+=1
        if airportData['ARPLongitudeS'][i][-1]=='W':
            west+=1
        else:
            east+=1
print("In the US, there're {} and {} airports with southern and northern latitudes respectively and {} and {} airports with western and eastern longitudes respectively.".format(south,north,west,east))

In the US, there're 4 and 13094 airports with southern and northern latitudes respectively and 13089 and 9 airports with western and eastern longitudes respectively.


In [6]:
connect = sqlite3.connect('./airportDistanceLookUp/database/airport.db')
c = connect.cursor()

def constructTable():
    print('-'*50)
    c.execute('CREATE TABLE IF NOT EXISTS Airport(ID INTEGER, LocationID TEXT, State TEXT, County TEXT, City TEXT, FacilityName TEXT, Latitude FLOAT, Longitude FLOAT)')
    print("Table Constructed: Airport")
    print('-'*50)
def valueInsertion():
    n = len(airportData)
    count = 1 
    for i in range(n):
        if airportData['Type'][i] == 'AIRPORT':
            _id = count
            locationID = airportData['LocationID'][i]
            locationID = locationID[1:]
            state = airportData['State'][i]
            county = airportData['County'][i]
            city = airportData['City'][i]
            facilityName = airportData['FacilityName'][i]
            latitude = airportData['ARPLatitudeS'][i]
            longitude = airportData['ARPLongitudeS'][i]
            if latitude[-1]=='S':
                latitude = float(latitude[:len(latitude)-1])*-1
            else:
                latitude = float(latitude[:len(latitude)-1])
            if longitude[-1]=='W':
                longitude = float(longitude[:len(longitude)-1])*-1
            else:
                longitude = float(longitude[:len(longitude)-1])
            c.execute('INSERT INTO Airport (ID,LocationID, State, County, City, FacilityName, Latitude,Longitude) VALUES (?,?,?,?,?,?,?,?)',
                     (_id,locationID,state,county,city,facilityName,latitude,longitude))
            connect.commit()
            count+=1
    print('Airport has been fully populated')
    print('-'*50)
def readFromFieldsDB():
    c.execute('SELECT * FROM Airport')
    data1 = c.fetchmany(size = 25)
    print('-'*50)
    print('First 25 rows in Airport:',*data1,sep='\n'+'-'*100+'\n',end = '\n'+'-'*100)

In [7]:
constructTable()
valueInsertion()

--------------------------------------------------
Table Constructed: Airport
--------------------------------------------------
Airport has been fully populated
--------------------------------------------------


In [8]:
c.execute('SELECT * FROM Airport')
print('-'*50)
print("There're {} entries in the Airport table.".format(len(c.fetchall())))
readFromFieldsDB()
c.close()
connect.close()

--------------------------------------------------
There're 13098 entries in the Airport table.
--------------------------------------------------
First 25 rows in Airport:
----------------------------------------------------------------------------------------------------
(1, 'ADK', 'AK', 'ALEUTIANS WEST', 'ADAK ISLAND', 'ADAK', 186780.8954, -635912.9277)
----------------------------------------------------------------------------------------------------
(2, 'AKK', 'AK', 'KODIAK ISLAND', 'AKHIOK', 'AKHIOK', 204979.287, -555057.2)
----------------------------------------------------------------------------------------------------
(3, 'Z13', 'AK', 'BETHEL', 'AKIACHAK', 'AKIACHAK', 219289.715, -581375.985)
----------------------------------------------------------------------------------------------------
(4, 'AKI', 'AK', 'BETHEL', 'AKIAK', 'AKIAK', 219250.43, -580430.1)
----------------------------------------------------------------------------------------------------
(5, '7AK', 'AK', 